In [1]:
import os
import sys
from pathlib import Path


def initialize_environment():
    BASE_DIR = str(Path("./../").resolve())
    if BASE_DIR not in sys.path:
        sys.path.append(BASE_DIR)


initialize_environment()
from analytics.strategies.sma_crossovers import SMAStrategy, Trend
from analytics.services.alpha_vantage import TimeInterval, AVTimeseries


def print_heading(text: str):
    print("###############################\n")
    print(text)
    print("\n###############################")


%load_ext nb_black

<IPython.core.display.Javascript object>

## Let us start off by looking up for the correct ticker symbol using Alpha vantage. 

In [7]:
# if you dont have an alpha vantage api_key get it from -
# https://www.alphavantage.co/support/#api-key
av_obj = AVTimeseries(api_key="50VORHAYXU7L3Q33")

search_keyword = "ibm"

<IPython.core.display.Javascript object>

In [9]:
search_symbol_results_df = av_obj.get_symbol_search_results(
    search_keyword=search_keyword
)
display(search_symbol_results_df)

,symbol,name,type,region,marketOpen,marketClose,timezone,currency,matchScore
0,IBM,International Business Machines Corporation,Equity,United States,09:30,16:00,UTC-05,USD,1.0000
1,IBMM,iShares iBonds Dec 2024 Term Muni Bond ETF,ETF,United States,09:30,16:00,UTC-05,USD,0.8571
2,IBMK,iShares iBonds Dec 2022 Term Muni Bond ETF,ETF,United States,09:30,16:00,UTC-05,USD,0.8571
3,IBMP,iShares iBonds Dec 2027 Term Muni Bond ETF,ETF,United States,09:30,16:00,UTC-05,USD,0.8571
4,IBMI,iShares iBonds Sep 2020 Term Muni Bond ETF,ETF,United States,09:30,16:00,UTC-05,USD,0.8571
5,IBMN,iShares iBonds Dec 2025 Term Muni Bond ETF,ETF,United States,09:30,16:00,UTC-05,USD,0.8571
6,IBMJ,iShares iBonds Dec 2021 Term Muni Bond ETF,ETF,United States,09:30,16:00,UTC-05,USD,0.8571
7,IBMO,iShares iBonds Dec 2026 Term Muni Bond ETF,ETF,United States,09:30,16:00,UTC-05,USD,0.8571
8,IBML,iShares iBonds Dec 2023 Term Muni Bond ETF,ETF,United States,09:30,16:00,UTC-05,USD,0.8571


<IPython.core.display.Javascript object>

### Pick the correct symbol from search results

* From here on, we shall use symbol 'IBM' (first row from the search results) to demonstrate computation further. 

In [10]:
symbol = search_symbol_results_df["symbol"].iloc[0]
print(symbol)

IBM


<IPython.core.display.Javascript object>

In [11]:
# Lets grab 60min intraday data for IBM
# supported time_intervals are ['ONE_MIN', 'FIVE_MIN', 'FIFTEEN_MIN', 'THIRTY_MIN', 'SIXTY_MIN']

av_intraday_df = av_obj.get_intraday_data(
    symbol=symbol, interval=TimeInterval.SIXTY_MIN
)
display(av_intraday_df.head())

,open,high,low,close,volume
2020-09-30 12:00:00,122.3300,122.7279,122.115,122.395,298034.0
2020-09-30 13:00:00,122.3883,122.7050,122.330,122.520,251536.0
2020-09-30 14:00:00,122.5100,122.7100,122.150,122.615,226438.0
2020-09-30 15:00:00,122.6300,122.9100,121.305,121.995,378808.0
2020-09-30 16:00:00,121.9600,122.1100,120.800,121.530,640914.0


<IPython.core.display.Javascript object>

In [12]:
# Similarly lets grab daily data for IBM using Alpha Vantage

av_daily_df = av_obj.get_daily_data(symbol=symbol)
display(av_daily_df.head())

,open,high,low,close,volume
2020-05-20,120.93,122.47,120.60,121.38,3840700.0
2020-05-21,120.99,121.72,118.97,119.12,4000900.0
2020-05-22,119.37,119.47,117.59,118.39,4180800.0
2020-05-26,121.30,122.62,121.01,121.76,5469300.0
2020-05-27,123.94,125.83,123.47,125.54,5727500.0


<IPython.core.display.Javascript object>

In [13]:
# Bottom up approach - Lets us evaluate how profitable 20-10 SMA cross-over strategy for IBM is. 
# It is same simple as pasisng the correct dataframe and defining the slower moving average time period 
# and faster moving time period. 
# SO lets get to it. 

# Ensure that slow_ma is greater than fast_ma. It makes logical sense. 
# If slow_ma < fast_ma, there will be an AssertionError
SMAStrategy.evaluate_sma_crossover(raw_scrip_df=av_intraday_df, slow_ma=20, fast_ma=10, capture_trend=Trend.BULLISH)


,,percent_returns,session_details
sma_session_20_10,label_20_10,,
1,bullish,-0.318862,2020-10-01 10:00:00-2020-10-01 11:00:00
3,bullish,-0.672627,2020-10-01 13:00:00-2020-10-01 18:00:00
5,bullish,1.846638,2020-10-02 18:00:00-2020-10-07 09:00:00
7,bullish,6.724969,2020-10-07 14:00:00-2020-10-09 09:00:00


<IPython.core.display.Javascript object>

### Top-to-Bottom approach 

* The SMAStrategy class provides helper functions to compute and evaluate numbers step by step. 

In [14]:
# define slow and fast moving average look back periods
# Again, ensure that slow_ma > fast_ma
slow_ma = 20
fast_ma = 10

# create an instance of SMAStrategy
sma_strategy_obj = SMAStrategy(scrip_df=av_intraday_df, slow_ma=20, fast_ma=10)


# compute simple moving averages
sma_df = sma_strategy_obj.compute_sma(look_back_periods=[10, 20])
print_heading("Simple Moving Averages")
display(sma_df)


# we can club the computation of SMA and annotattion using the sma_sessions method as well
sma_sessions_annotated_df = sma_strategy_obj.sma_sessions()
print_heading("Simple Moving Average Cross Over Strategy Annotated Sessions")
display(sma_sessions_annotated_df)

# use annotated data to get the summary of returns
column_suffix = f"{slow_ma}_{fast_ma}"
summary_df = sma_sessions_annotated_df.groupby(
    [f"sma_session_{column_suffix}", f"label_{column_suffix}"], as_index=True
).apply(SMAStrategy.compute_returns, slow_ma=slow_ma, fast_ma=fast_ma)
print_heading("Returns Summary")
# Note that the summary_df results will match the one from evaluate_sma_crossover classmathod.
display(summary_df)

###############################

Simple Moving Averages

###############################


,sma_10,sma_20
2020-09-30 12:00:00,122.3950,122.39500
2020-09-30 13:00:00,122.5200,122.52000
2020-09-30 14:00:00,122.6150,122.61500
2020-09-30 15:00:00,121.9950,121.99500
2020-09-30 16:00:00,121.5300,121.53000
...,...,...
2020-10-09 16:00:00,129.5975,130.36827
2020-10-09 17:00:00,129.1455,130.22502
2020-10-09 18:00:00,128.7355,130.08700
2020-10-09 19:00:00,128.3405,129.95450


###############################

Simple Moving Average Cross Over Strategy Annotated Sessions

###############################


,open,high,low,close,volume,sma_20,sma_10,sma_signal_20_10,sma_session_20_10,label_20_10
2020-09-30 12:00:00,122.3300,122.7279,122.115,122.395,298034.0,122.39500,122.3950,False,0,bearish
2020-09-30 13:00:00,122.3883,122.7050,122.330,122.520,251536.0,122.52000,122.5200,False,0,bearish
2020-09-30 14:00:00,122.5100,122.7100,122.150,122.615,226438.0,122.61500,122.6150,False,0,bearish
2020-09-30 15:00:00,122.6300,122.9100,121.305,121.995,378808.0,121.99500,121.9950,False,0,bearish
2020-09-30 16:00:00,121.9600,122.1100,120.800,121.530,640914.0,121.53000,121.5300,False,0,bearish
...,...,...,...,...,...,...,...,...,...,...
2020-10-09 16:00:00,127.9300,128.4999,127.600,127.810,1450088.0,130.36827,129.5975,False,8,bearish
2020-10-09 17:00:00,127.7900,127.8028,127.620,127.790,44924.0,130.22502,129.1455,False,8,bearish
2020-10-09 18:00:00,127.7900,127.8000,127.620,127.800,5440.0,130.08700,128.7355,False,8,bearish
2020-10-09 19:00:00,127.8000,127.8500,127.800,127.850,3619.0,129.95450,128.3405,False,8,bearish


###############################

Returns Summary

###############################


,,percent_returns,session_details
sma_session_20_10,label_20_10,,
0,bearish,-0.403204,2020-09-30 12:00:00-2020-10-01 09:00:00
1,bullish,-0.318862,2020-10-01 10:00:00-2020-10-01 11:00:00
2,bearish,0.000000,2020-10-01 12:00:00-2020-10-01 12:00:00
3,bullish,-0.672627,2020-10-01 13:00:00-2020-10-01 18:00:00
4,bearish,0.356639,2020-10-01 19:00:00-2020-10-02 17:00:00
5,bullish,1.846638,2020-10-02 18:00:00-2020-10-07 09:00:00
6,bearish,-0.428595,2020-10-07 10:00:00-2020-10-07 13:00:00
7,bullish,6.724969,2020-10-07 14:00:00-2020-10-09 09:00:00
8,bearish,1.391930,2020-10-09 10:00:00-2020-10-09 20:00:00


<IPython.core.display.Javascript object>